# --- Day 20: Jurassic Jigsaw ---

https://adventofcode.com/2020/day/20

In [1]:
path = '../inputs/'

In [2]:
import re

## Part 1

In [3]:
def read_tiles(file_name):
    tiles = {}
    
    with open(path + file_name) as file:
        tile_data = file.read().split('\n\n')

    for tile in tile_data:
        id = int(tile[5:9])

        t = tile.split('\n')
        t.pop(0) # Get rid of tile ID row

        tiles[id] = {}
        tiles[id]['top'] = t[0]
        tiles[id]['bottom'] = t[-1]

        _left = []
        _right = []
        
        for row in t:
            _left.append(row[0])
            _right.append(row[-1])

        tiles[id]['left'] = ''.join(_left)
        tiles[id]['right'] = ''.join(_right)
    
    return tiles

In [4]:
def find_corners(tiles):
    
    corners = []
    
    for i in tiles.keys():
        shared_sides = 0
        for j in tiles.keys():
            if i != j:
                j_sides = [
                    tiles[j]['top'],
                    tiles[j]['top'][::-1], # Reversed top
                    tiles[j]['right'],
                    tiles[j]['right'][::-1], # Reversed right
                    tiles[j]['bottom'], 
                    tiles[j]['bottom'][::-1], # Reversed bottom
                    tiles[j]['left'],
                    tiles[j]['left'][::-1], # Reversed left
                ]
                
                if tiles[i]['top'] in j_sides: 
                    shared_sides += 1
                if tiles[i]['right'] in j_sides:
                    shared_sides += 1
                if tiles[i]['bottom'] in j_sides:
                    shared_sides += 1
                if tiles[i]['left'] in j_sides:
                    shared_sides += 1
        
        if shared_sides == 2:
            corners.append(i)
    
    return corners

In [5]:
def multiply_corners(corners):
    total = 1
    for c in corners:
        total *= c

    return total

In [6]:
tiles = read_tiles('example_tiles.txt')
corners = find_corners(tiles)
print(corners)
multiply_corners(corners) # Should return 20899048083289

[1951, 1171, 2971, 3079]


20899048083289

In [7]:
tiles = read_tiles('tiles.txt')
corners = find_corners(tiles)
print(corners)
multiply_corners(corners)

[1049, 2081, 2129, 3229]


15006909892229

## Part 2

### Set up data structure

In [8]:
def get_rotations(data):
    """Return list of all 4 cardinal rotations of the image/tile data."""
    rotated_90 = [''.join(list(row)) for row in zip(*data[::-1])]
    rotated_180 = [''.join(list(row)) for row in zip(*rotated_90[::-1])]
    rotated_270 = [''.join(list(row)) for row in zip(*data)][::-1]
    
    return [data, rotated_90, rotated_180, rotated_270]

In [9]:
def flip(data):
    """Flip the image/tile data along the y-axis (each row is reversed)."""
    flipped = data.copy()
    
    for i, _ in enumerate(flipped):
        flipped[i] = flipped[i][::-1]
    
    return flipped

In [10]:
def get_edges(tile_data):
    """Return a dictionary of the edges from from tile_data."""
    edges = {}

    edges['top'] = tile_data[0]
    edges['bottom'] = tile_data[-1]
    edges['left'] = ''.join([row[0] for row in tile_data])
    edges['right'] = ''.join([row[-1] for row in tile_data])
    
    return edges

In [11]:
def read_tiles(file_name):
    """Read in tiles data, and created necessary data structures."""
    tiles = {}
    
    with open(path + file_name) as file:
        tile_data = file.read().split('\n\n')

    for tile in tile_data:
        id = int(tile[5:9])

        t = tile.split('\n')
        t.pop(0) # Get rid of tile id row

        tiles[id] = {}
        tiles[id]['data'] = None  # This will hold the data for the appropriate orientation
        tiles[id]['orientation'] = None # 0:7 -- 4 rotations and 4 flipped rotations
        
        tiles[id]['neighbors'] = {}
        tiles[id]['neighbors']['top'] = None
        tiles[id]['neighbors']['right'] = None
        tiles[id]['neighbors']['bottom'] = None
        tiles[id]['neighbors']['left'] = None
        
        rotations = get_rotations(t) + get_rotations(flip(t))
        
        for i, rotation in enumerate(rotations):
            tiles[id][i] = {}
            tiles[id][i]['data'] = rotation
            tiles[id][i]['edges'] = get_edges(rotation)
            
    return tiles

In [12]:
# Test
# tiles = read_tiles('example_tiles.txt')
# tiles[2311]

### Find neighbors

In [13]:
def find_neighbors(tile_id_list, orient_list, tiles):
    """Recursive function to find the top, right, bottom, and left neighbors for each tile."""
    
    orientations = range(8)  # There are 8 possible orientations of a tile: 4 rotations + 4 rotations(of a flipped tile)
    sides = ['top', 'right', 'bottom', 'left']
    good_match = {'top' : 'bottom',
                  'bottom' : 'top',
                  'left' : 'right',
                  'right' : 'left'}
    
    # base case
    if len(tile_id_list) == 0:
        return tiles
    
    # recursive case
    else:
        i = tile_id_list.pop(0)
        orient_i = orient_list.pop(0)
        tiles[i]['orientation'] = orient_i
        other_keys = [k for k in tiles.keys() if k != i] # Remove i from list of keys to loop over    
        
        for side_i in sides:

            for j in other_keys:
                
                side_j = good_match[side_i]
                
                if tiles[j]['orientation'] != None:
                    iter_j = [tiles[j]['orientation']]
                else:
                    iter_j = orientations

                for orient_j in iter_j:

                    if j not in tiles[i]['neighbors'].values():
                        
                        if tiles[i][orient_i]['edges'][side_i] == tiles[j][orient_j]['edges'][side_j]:

                            tiles[j]['neighbors'][side_j] = i
                            tiles[i]['neighbors'][side_i] = j

                            tiles[j]['orientation'] = orient_j
                            tiles[j]['data'] = tiles[j][orient_j]['data']
                            tiles[i]['data'] = tiles[i][orient_i]['data']
                            
                            tile_id_list.append(j)
                            orient_list.append(orient_j)

                            break

        return find_neighbors(tile_id_list, orient_list, tiles)

In [14]:
# Test
tiles = read_tiles('example_tiles.txt')

tile_id_list = [list(tiles)[0]]
orient_list = [6] # This is the orientation for the first tile (2311) in the example

tiles = find_neighbors(tile_id_list, orient_list, tiles)

for key in tiles:
    print(key, tiles[key]['orientation'], tiles[key]['neighbors'])

2311 6 {'top': None, 'right': 3079, 'bottom': 1427, 'left': 1951}
1951 6 {'top': None, 'right': 2311, 'bottom': 2729, 'left': None}
1171 4 {'top': 2473, 'right': None, 'bottom': None, 'left': 1489}
1427 6 {'top': 2311, 'right': 2473, 'bottom': 1489, 'left': 2729}
1489 6 {'top': 1427, 'right': 1171, 'bottom': None, 'left': 2971}
2473 5 {'top': 3079, 'right': None, 'bottom': 1171, 'left': 1427}
2971 6 {'top': 2729, 'right': 1489, 'bottom': None, 'left': None}
2729 6 {'top': 1951, 'right': 1427, 'bottom': 2971, 'left': None}
3079 0 {'top': None, 'right': None, 'bottom': 2473, 'left': 2311}


## Make Image

In [15]:
def remove_edges(tile_data):
    """Remove the edges on a tile of data."""
    data_wo_edges = tile_data[1:-1]
    
    for i, _ in enumerate(data_wo_edges):
        data_wo_edges[i] = data_wo_edges[i][1:-1]
    
    return data_wo_edges

In [16]:
def get_row(tile_id, tiles):
    """Recursive function to get a list of all the concatenated strings for each tile in a row."""
    
    row = remove_edges(tiles[tile_id]['data'].copy())
    right = tiles[tile_id]['neighbors']['right']
    
    if right is None:
        return row
    
    else:
        row = [i + j for i, j in zip(row, get_row(right, tiles))]
    
    return row

In [17]:
# Test
# get_row(1951, tiles)  # tile 1951 is the top-left tile

In [18]:
def get_col(tile_id, tiles):
    """Recursive function to get a list of all the tiles in a column, starting with the top-left tile_id."""

    col = [tile_id]
    bottom = tiles[tile_id]['neighbors']['bottom']
    
    if bottom is None:
        return col
    
    else:
        col += get_col(bottom, tiles)
    
    return col

In [19]:
# Test
# get_col(1951, tiles)

In [20]:
def make_image(tiles):
    """Make the image from the tiles (with edges removed)."""

    image = []

    # Get top left tile to start
    for key in tiles.keys():
        if tiles[key]['neighbors']['top'] == None and tiles[key]['neighbors']['left'] == None:
            top_left_tile = key
            break
    
    for r in get_col(top_left_tile, tiles):
        image += (get_row(r, tiles)) 

    return image

In [21]:
# Test
# make_image(tiles)

### Define regular expressions for the sea monster

In [22]:
head_re = re.compile('..................#.')
body_re = re.compile('#....##....##....###')
feet_re = re.compile('.#..#..#..#..#..#...')
num_pounds_in_sea_monster = 15

In [23]:
def count_pounds(image_data):
    """Count the number of pound symbols in the image."""
    
    num_pounds = 0
    pound_re = re.compile('#')
    
    for row in image_data:
        num_pounds += len(pound_re.findall(row))
    
    return num_pounds

In [24]:
def find_sea_monsters(image):
    """Return the number of sea monsters found in the image."""

    num_sea_monsters = 0
    
    for i, _ in enumerate(image):

        match = body_re.finditer(image[i])
        
        for m in match:
            start_pos = m.start()
            end_pos = m.end()

            if head_re.search(image[i-1], start_pos, end_pos) \
              and feet_re.search(image[i+1], start_pos, end_pos):
                num_sea_monsters += 1

    return num_sea_monsters

## Test search

In [25]:
with open(path + 'example_image_with_2_sea_monsters.txt') as file:
    example_image = file.read().splitlines()
example_image

['.####...#####..#...###..',
 '#####..#..#.#.####..#.#.',
 '.#.#...#.###...#.##.##..',
 '#.#.##.###.#.##.##.#####',
 '..##.###.####..#.####.##',
 '...#.#..##.##...#..#..##',
 '#.##.#..#.#..#..##.#.#..',
 '.###.##.....#...###.#...',
 '#.####.#.#....##.#..#.#.',
 '##...#..#....#..#...####',
 '..#.##...###..#.#####..#',
 '....#.##.#.#####....#...',
 '..##.##.###.....#.##..#.',
 '#...#...###..####....##.',
 '.#.##...#.##.#.#.###...#',
 '#.###.#..####...##..#...',
 '#.###...#.##...#.######.',
 '.###.###.#######..#####.',
 '..##.#..#..#.#######.###',
 '#.#..##.########..#..##.',
 '#.#####..#.#...##..#....',
 '#....##..#.#########..##',
 '#...#.....#..##...###.##',
 '#..###....##.#...##.##.#']

In [26]:
count_pounds(example_image) - (find_sea_monsters(example_image) * num_pounds_in_sea_monster) # Should return 273

273

## Put it all together and solve

In [27]:
from pprint import pprint
def solve(file_name):
    tiles = read_tiles(file_name)
    
    tile_id_list = [list(tiles)[0]]  # Start with first tile in the dictionary
    orient_list = [0]  # Start first tile with an arbitrary orientation
    
    tiles = find_neighbors(tile_id_list, orient_list, tiles)
               
    image = make_image(tiles)

    orientations = get_rotations(image) + get_rotations(flip(image))
        
    for orient in orientations:
        num_sea_monsters = find_sea_monsters(orient)
        if num_sea_monsters > 0:
            break

    return count_pounds(image) - (num_sea_monsters * num_pounds_in_sea_monster)

In [28]:
solve('example_tiles.txt') # Should return 273

273

In [29]:
solve('tiles.txt')

2190